In [161]:
%matplotlib inline
import json
import matplotlib.pyplot as plt
import pandas as pd
from pandas.io.json import json_normalize
import time
import numpy as np
import seaborn as sns
import re

In [162]:
path = "executions-bkp-gcloudeu-28-11.jsonl"
raw_data = pd.read_json(path, lines=True)
raw_data.head()

,_id,topic,payload,timestamp
0,{'$oid': '5d274d843299fa6236b8bc28'},data/case/332f02b9-093e-4a2f-8d93-28022c275e8b...,{'responseStamp': 1},2019-07-11 14:53:56.866
1,{'$oid': '5d274d853299fa6236b8bc29'},data/case/332f02b9-093e-4a2f-8d93-28022c275e8b...,"{'name': 'Zombie Venom 09', 'source': '# 1 (ca...",2019-07-11 14:53:57.587
2,{'$oid': '5d274e633299fa6236b8bc2a'},data/case/00f14675-854c-4c58-95eb-8dd414f525d3...,{'responseStamp': 1},2019-07-11 14:57:39.979
3,{'$oid': '5d274e643299fa6236b8bc2b'},data/case/00f14675-854c-4c58-95eb-8dd414f525d3...,"{'name': 'Zombie Venom 08', 'source': '# 1 (ca...",2019-07-11 14:57:40.069
4,{'$oid': '5d274e643299fa6236b8bc2c'},data/theme/zombie.note/get,{'responseStamp': 2},2019-07-11 14:57:40.085


In [163]:
nl_experiment = raw_data.loc[lambda df: df.timestamp > '2019-11-28 7:30:00']
nl_experiment.to_json('nl_experiment1.json', orient='records')

In [164]:
def reconstruct_users_track(log):
    users_tracks = dict()
    for index, row in log.iterrows():
        payload = row.payload
        if "track" in payload and "userid" in payload['track']:
            user = payload['track']['userid'] 
            if user in users_tracks:
                users_tracks[user].append(row)
            else:
                users_tracks.update({user: [row]})
    return users_tracks

In [165]:
def dic_to_dataFrames(dictionary):
    data_frames = []
    for key in dictionary.keys():
        df = pd.DataFrame(dictionary[key],columns=[key])
        data_frames.append(df)

    return pd.concat(data_frames, axis=1)

In [166]:
nl_experiment.head(10)

,_id,topic,payload,timestamp
22113,{'$oid': '5ddf8655cd683579f92534ee'},data/user/login,"{'email': 'A33@ds4h.org', 'password': 'a33', '...",2019-11-28 08:33:25.387
22114,{'$oid': '5ddf8655cd683579f92534ef'},data/user/login/response/2,"{'userid': 93, 'token': 'eyJhbGciOiJIUzI1NiIsI...",2019-11-28 08:33:25.551
22115,{'$oid': '5ddf8655cd683579f92534f0'},data/case/*/list,"{'filterBy': 'user', 'filter': 93, 'responseSt...",2019-11-28 08:33:25.614
22116,{'$oid': '5ddf8655cd683579f92534f1'},data/case/*/list/response/3,[{'id': 'a46c78ec-9973-4792-a4ec-d9a9ef0e4978'...,2019-11-28 08:33:25.729
22117,{'$oid': '5ddf8655cd683579f92534f2'},data/case/a46c78ec-9973-4792-a4ec-d9a9ef0e4978...,{'responseStamp': 4},2019-11-28 08:33:25.754
22118,{'$oid': '5ddf8655cd683579f92534f3'},data/case/a46c78ec-9973-4792-a4ec-d9a9ef0e4978...,"{'name': 'EKG-First', 'source': 'EKG-Descripti...",2019-11-28 08:33:25.883
22119,{'$oid': '5ddf8656cd683579f92534f4'},knot/<</navigate,{},2019-11-28 08:33:26.004
22120,{'$oid': '5ddf8656cd683579f92534f5'},2019-11-28T08:33:25.342Z-980cae4f-15d1-9d08-11...,"{'parameter': 'EKG first', 'track': {'userid':...",2019-11-28 08:33:26.033
22121,{'$oid': '5ddf8656cd683579f92534f6'},2019-11-28T08:33:25.342Z-980cae4f-15d1-9d08-11...,"{'track': {'userid': 93, 'caseid': 'a46c78ec-9...",2019-11-28 08:33:26.036
22122,{'$oid': '5ddf8656cd683579f92534f7'},2019-11-28T08:33:25.342Z-980cae4f-15d1-9d08-11...,"{'track': {'userid': 93, 'caseid': 'a46c78ec-9...",2019-11-28 08:33:26.082


In [167]:
users_track = reconstruct_users_track(nl_experiment)

In [168]:
users_track[93][0]['payload']['parameter']

'EKG first'

In [169]:
def split_groups(users_track):
    group_A = dict()
    group_B = dict()
    
    for user in users_track.keys():
        for log in users_track[user]:
            try:
                if log['payload']['parameter'] == 'Description first':
                    group_A[user] = users_track[user]
                if log['payload']['parameter'] == 'EKG first':
                    group_B[user] = users_track[user]
            except:
                pass
    return group_A, group_B

In [170]:
group_A, group_B = split_groups(users_track)
print(sorted(list(group_A.keys())))
print(sorted(list(group_B.keys())))

[1, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87]
[86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 115, 116]


In [171]:
print(len(list(group_A.keys())))
print(len(list(group_B.keys())))

26
29


In [172]:
def users_answers(users_track):
    users_answers = dict()
    for user in users_track.keys():
        for log in users_track[user]:
            if re.search('/var/[\w+.]+/set', log['topic']):
                answer=log['payload']['content']
                if user in users_answers:
                    users_answers[user].append(answer)
                else:
                    users_answers[user] = []
            if '/control/input/submit' in log['topic']:
                if user in users_answers:
                    users_answers[user].append('submit')
                else:
                    users_answers[user] = []
    return users_answers

In [173]:
answers_groupA = users_answers(group_A)
answers_groupA[61]

['submit',
 'Acs',
 'submit',
 '60',
 'Acs',
 '60',
 'submit',
 'Acs',
 '60',
 'Nstemi',
 '85',
 'submit',
 'Acs',
 '60',
 '85',
 'Nstemi',
 'submit',
 'submit',
 'Gastrointestinal',
 '85',
 'submit',
 'Gastrointestinal',
 '85',
 'submit',
 'Gastrointestinal',
 '85',
 'Gastrointestinal',
 '65',
 'submit',
 'Gastrointestinal',
 '85',
 '65',
 'Gastrointestinal',
 'submit',
 'submit',
 'Tendomyogenic',
 '85',
 'submit',
 'Tendomyogenic',
 '85',
 'submit',
 '85',
 'Tendomyogenic',
 'Tendomyogenic',
 '85',
 'submit',
 'Tendomyogenic',
 '85',
 'Tendomyogenic',
 '85',
 'submit',
 'submit',
 'Pericarditis',
 '50',
 'submit',
 'Pericarditis',
 '50',
 'submit',
 'Pericarditis',
 '50',
 'Pericarditis',
 '91',
 'submit',
 'Pericarditis',
 '91',
 'Pericarditis',
 'submit',
 '50',
 'submit',
 'Acs',
 '70',
 'submit',
 'Acs',
 'submit',
 '70',
 '70',
 'Acs',
 'Acs',
 '90',
 'submit',
 'Acs',
 '70',
 'Acs',
 '90',
 'submit',
 'submit',
 'Acs',
 '75',
 'submit',
 'Acs',
 '75',
 'submit',
 '75',
 'Acs',

In [174]:
l = answers_groupA[61]
sum(i == 'submit' for i in l)

60